In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
FILE_MODEL_NAME = "bible_model.sav"
DATA_FILE_NAME = "bible.txt"
TWEETS_FILE_NAME = "tweets_storage.txt"
NR_OF_TRAINING_CHARACTERS = 1000000
NR_UNITS = 300
def load_data():

    # load text
    filename = DATA_FILE_NAME
    text = (open(filename).read())

    tweets_file_name = TWEETS_FILE_NAME
    text = (open(filename).read()) + "\n" + text

    text = text[0:NR_OF_TRAINING_CHARACTERS]
    text = text.lower()
    # print(text)

    # mapping characters with integers
    unique_chars = sorted(list(set(text)))

    char_to_int = {}
    int_to_char = {}

    for i, c in enumerate(unique_chars):
        char_to_int.update({c: i})
        int_to_char.update({i: c})

    # print(char_to_int)
    # print(int_to_char)

    # preparing input and output dataset
    X = []
    Y = []
    number_period = 50

    for i in range(0, len(text) - number_period, 1):
        sequence = text[i:i + number_period]
        label = text[i + number_period]
        X.append([char_to_int[char] for char in sequence])
        Y.append(char_to_int[label])

    # print("X", X)
    # print("Y", Y)

    # reshaping, normalizing and one hot encoding
    X_modified = np.reshape(X, (len(X), number_period, 1))
    # print("X_modified", X_modified)
    X_modified = X_modified / float(len(unique_chars))
    # print("X_modified", X_modified)
    Y_modified = np_utils.to_categorical(Y)
    # print("Y_modified", Y_modified)

    return unique_chars, int_to_char, char_to_int, X, Y, X_modified, Y_modified

In [4]:
def train_model(X_modified, Y_modified, load_checkpoint=False):

    # defining the LSTM model
    model = Sequential()
    model.add(LSTM(NR_UNITS, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(NR_UNITS))
    model.add(Dropout(0.2))
    model.add(Dense(Y_modified.shape[1], activation='softmax'))

    # Checkpoint loading
    if load_checkpoint:
        model.load_weights("bible_weights_improvement.hdf5")

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    epoch_number = 8

    # Checkpointing
    filepath = "bible_weights_improvement.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    # fitting the model
    model.fit(X_modified, Y_modified, epochs=epoch_number, batch_size=16, validation_split=0.33, callbacks=callbacks_list)

    filename = FILE_MODEL_NAME
    save_model(filename, model)

In [ ]:
# Train model

print("Loading data")
unique_chars, int_to_char, char_to_int, X, Y, X_modified, Y_modified = load_data()
print("Training began")
train_model(X_modified, Y_modified, load_checkpoint = False)

Loading data
